In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import pandas as pd
import OAPBuoyData as OAP, commonfxns as cf, diagsPP, viz
import cftime
import warnings

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [2]:
cmip6data='/space/hall6/sitestore/eccc/crd/ccrn/model_output/CMIP6/final/CMIP6/'
h5ss='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/'
canesm5assim='/space/hall5/sitestore/eccc/crd/ccrn/model_output/CMIP6/final/CMIP6/DCPP/CCCma/CanESM5/dcppA-assim/'

In [3]:
def originpath(ivar,scen,fdir,ver='v20190429'):
    if scen in ('historical',):
        return os.path.join(cmip6data,'CMIP/CCCma/CanESM5-CanOE',scen,'r1i1p2f1',fdir,ivar,'gn',ver)
    elif scen in ('ssp245',):
        return os.path.join(cmip6data,'ScenarioMIP/CCCma/CanESM5-CanOE',scen,'r1i1p2f1',fdir,ivar,'gn',ver)

In [4]:
def getfiles(opath):
    return os.listdir(opath)

In [5]:
flist={}
for ivar in ('tos','sos','chlos','phos','spco2','o2os','o2satos','no3os','limndiat','limnmisc','intpp','phydiatos','phymiscos'):
    flist[ivar]=[]
    for ff in getfiles(originpath(ivar,'historical','Omon')):
        if not os.path.exists(os.path.join(h5ss,'work',ff)):
            shutil.copy(os.path.join(originpath(ivar,'historical','Omon'),ff),os.path.join(h5ss,'work'))
        flist[ivar].append(os.path.join(h5ss,'work',ff))
    for ff in getfiles(originpath(ivar,'ssp245','Omon')):
        if not os.path.exists(os.path.join(h5ss,'work',ff)):
            shutil.copy(os.path.join(originpath(ivar,'ssp245','Omon'),ff),os.path.join(h5ss,'work'))
        flist[ivar].append(os.path.join(h5ss,'work',ff))

In [6]:
flist

{'tos': ['/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/tos_Omon_CanESM5-CanOE_historical_r1i1p2f1_gn_185001-201412.nc',
  '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/tos_Omon_CanESM5-CanOE_ssp245_r1i1p2f1_gn_201501-210012.nc'],
 'sos': ['/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/sos_Omon_CanESM5-CanOE_historical_r1i1p2f1_gn_185001-201412.nc',
  '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/sos_Omon_CanESM5-CanOE_ssp245_r1i1p2f1_gn_201501-210012.nc'],
 'chlos': ['/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/chlos_Omon_CanESM5-CanOE_historical_r1i1p2f1_gn_185001-201412.nc',
  '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/chlos_Omon_CanESM5-CanOE_ssp245_r1i1p2f1_gn_201501-210012.nc'],
 'phos': ['/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/phos_Omon_CanESM5-CanOE_historical_r1i1p2f1_gn_185001-201412.nc',
  '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/phos_Omon_CanESM5-CanOE_ssp245_r1i1p2f1_gn_201501-2100

In [7]:
dfb=OAP.loadOAPInfo(modelgrid=True)

In [8]:
dfb

,datasetID,title,minTime,maxTime,Lat,Lon,tabfile,jj,ii,Lat1x1,Lon1x1,jjHD,iiHD,modBathy,modBathyHD,shortTitle
index,,,,,,,,,,,,,,,,
0,pmel_co2_moorings_ba50_a09b_5565,Iceland NOAA Surface Ocean CO2 and Ocean Acid...,2013-08-16T10:47:00Z,2017-06-28T18:17:00Z,68.000,-12.6700,ICELAND.txt,248,270,68.280304,347.213867,248,270,1812.624146,1812.624146,Iceland
1,pmel_co2_moorings_c2e7_ecb9_4565,Gulf of Maine NOAA Surface Ocean CO2 and Ocea...,2006-07-13T02:16:00Z,2021-06-02T09:17:00Z,43.023,-70.5420,GULFOFMAINE.txt,209,216,42.513798,289.199371,209,216,52.952694,52.952694,Gulf of Maine
2,pmel_co2_moorings_c64b_e2bb_dc32,First Landing NOAA Surface Ocean CO2 and Ocea...,2018-04-05T21:17:00Z,2020-09-26T15:17:00Z,36.998,-76.0880,FIRSTLANDING.txt,201,211,36.721848,284.420410,201,211,21.984417,21.984417,First Landing
3,pmel_co2_moorings_4e4c_1834_1204,Hog Reef NOAA Surface Ocean CO2 and Ocean Aci...,2010-12-05T03:17:00Z,2018-07-31T12:16:00Z,32.460,-64.8300,HOGREEF.txt,195,222,32.074780,295.478455,195,222,4167.953613,4167.953613,Hog Reef
4,pmel_co2_moorings_e1ef_470f_1fee,Crescent Reef NOAA Surface Ocean CO2 and Ocea...,2010-11-27T03:17:00Z,2015-07-07T15:17:00Z,32.400,-64.7900,CRESCENTREEF.txt,195,222,32.074780,295.478455,195,222,4167.953613,4167.953613,Crescent Reef
5,pmel_co2_moorings_939b_fc25_57be,BTM NOAA Surface Ocean CO2 and Ocean Acidific...,2005-10-22T00:16:00Z,2007-10-01T12:16:00Z,31.500,-64.2000,BTM.txt,194,222,31.240679,295.483887,194,222,4504.969238,4504.969238,BTM
6,pmel_co2_moorings_ab27_2faf_3aa1,Gray's Reef NOAA Surface Ocean CO2 and Ocean ...,2006-07-18T00:45:00Z,2018-08-27T12:17:00Z,31.400,-80.8700,GRAYSREEF.txt,194,206,31.175501,279.486389,194,206,26.980017,26.980017,Gray's Reef
7,pmel_co2_moorings_cf08_f32f_712a,Coastal MS NOAA Surface Ocean CO2 and Ocean A...,2009-05-12T18:17:00Z,2017-05-29T21:17:00Z,30.000,-88.6000,COASTALMS.txt,193,198,30.324305,271.492432,193,198,20.030680,20.030680,Coastal MS
8,pmel_co2_moorings_0af7_8b89_77bd,Coastal LA NOAA Surface Ocean CO2 and Ocean A...,2017-07-14T18:46:00Z,2020-08-12T21:17:00Z,28.520,-90.2878,COASTALLA.txt,191,196,28.635588,269.496643,191,196,26.980017,26.980017,Coastal LA


In [9]:
yrspan=[2014,2022]
name='CanESM5CanOE_1975_2022_monthly'

In [10]:
warnings.filterwarnings(action='ignore',message='The specified chunks')

In [11]:
indlist=[]
for ivar in flist.keys():
    print(ivar)
    flong=xr.open_mfdataset(flist[ivar],parallel=True,chunks={'time':-1,'j':5,'i':5}).sel(time=slice(cftime.DatetimeNoLeap(1975,1,1),cftime.DatetimeNoLeap(2023,1,1)), )
    print('loaded')
    for ind, row in dfb.iterrows():
        if row.modBathy>0:
            fnameout=diagsPP.extractedTSPath(name,row.datasetID,row.jj,row.ii)
            if ivar=='tos' and os.path.exists(fnameout): os.remove(fnameout)
            opt='a' if os.path.exists(fnameout) else 'w'
            cf.mkdirs(fnameout)
            print(fnameout)
            indlist.append(ind)
            flongi=flong.isel(j=row.jj,i=row.ii)
            flongi.to_netcdf(fnameout,opt)
            del flongi 
    flong.close()
print('done')

tos
loaded
/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/extracted/CanESM5CanOE_1975_2022_monthly/CanESM5CanOE_1975_2022_monthly.pmel_co2_moorings_ba50_a09b_5565.j248i270_1x1grid.nc
/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/extracted/CanESM5CanOE_1975_2022_monthly/CanESM5CanOE_1975_2022_monthly.pmel_co2_moorings_c2e7_ecb9_4565.j209i216_1x1grid.nc
/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/extracted/CanESM5CanOE_1975_2022_monthly/CanESM5CanOE_1975_2022_monthly.pmel_co2_moorings_c64b_e2bb_dc32.j201i211_1x1grid.nc
/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/extracted/CanESM5CanOE_1975_2022_monthly/CanESM5CanOE_1975_2022_monthly.pmel_co2_moorings_4e4c_1834_1204.j195i222_1x1grid.nc
/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/extracted/CanESM5CanOE_1975_2022_monthly/CanESM5CanOE_1975_2022_monthly.pmel_co2_moorings_e1ef_470f_1fee.j195i222_1x1grid.nc
/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/extracted/CanESM5CanOE_1975_2022_m